In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/transl-last-hour/translator1/fingerprint.pb
/kaggle/input/transl-last-hour/translator1/saved_model.pb
/kaggle/input/transl-last-hour/translator1/variables/variables.index
/kaggle/input/transl-last-hour/translator1/variables/variables.data-00000-of-00001
/kaggle/input/all-data-vocab-53k-mixed-batch/translator1/fingerprint.pb
/kaggle/input/all-data-vocab-53k-mixed-batch/translator1/saved_model.pb
/kaggle/input/all-data-vocab-53k-mixed-batch/translator1/variables/variables.index
/kaggle/input/all-data-vocab-53k-mixed-batch/translator1/variables/variables.data-00000-of-00001
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/fingerprint.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/saved_model.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.index
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.data-00000-of-00001
/kaggle/input/final-20-01-15e/translator1/fingerprint.pb
/kaggle/input/final-20-

In [2]:
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

In [5]:
!pip install Metaphone
from metaphone import doublemetaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=b3f7923c45b9c42fe1d543e41cce2cd2383c83150cd35789ca0253e3ab7f9809
  Stored in directory: /root/.cache/pip/wheels/3a/2c/57/cb7d38aa6c80a870a0341ccf4265c2921fb55b1f7f4b76b46b
Successfully built Metaphone


In [6]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [7]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [8]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [9]:
import pandas as pd
data_val=pd.read_csv('/kaggle/input/validate-bleu/validation.tsv',sep='\t')
data_val=data_val[['cs_query','en_query']]
data_val=data_val.rename(columns={'cs_query':'Sentence','en_query':'trr'})
data_val['trr']=data_val['trr'].apply(lambda x: x.lower().strip())
data_val

,Sentence,trr
0,mere timer ko roko,pause my timer .
1,Please Liz ke liye video message banaye,please make video message for liz
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai,library events in cedar park
4,Please timer ko rokey,please pause timer
...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...
1387,please Tupac play kare,please play tupac
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...


In [10]:
'''three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.translate(input_text=three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())'''

"three_input_text = tf.constant([t_text('sabse best friend'), \n                                t_text('That toh I know'),\n                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),\n                                t_text('tum batao'),\n                                t_text('kya karna hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhara papa ka naam kya hai'),\n])\n\nresult = reloaded.translate(input_text=three_input_text)\n\nfor tr in result['text']:\n  print(tr.numpy().decode())"

In [11]:
import numpy

In [12]:
def cal_bleu(x,y):
    return sentence_bleu([x.split()],y.split())

In [18]:
reloaded = tf.saved_model.load('/kaggle/input/transl-last-hour/translator1')
data_val['model_output1']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu(*x),axis=1)
#data_val['bleu_score_inv1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu_inv(*x),axis=1)

In [19]:
data_val

,Sentence,trr,model_output17,bleu_score17,model_output18,bleu_score18,model_output1,bleu_score1
0,mere timer ko roko,pause my timer .,pause my timer for an allowing it is filled !,0.302138,pause my timer with a naptime,0.472871,pause my timer .,1.000000
1,Please Liz ke liye video message banaye,please make video message for liz,please rai of liz s started moving .,0.707107,please rai of the liz s parma to message .,0.740083,please give liz s happiness turned on be a mem...,0.652994
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,create alarms at pm and pm on tuesday,0.343645,create an alarm on tuesday and pm on tuesday,0.462306,alarm sounds at pm and pm on tuesday .,0.361774
3,Library events Cedar Park me hai,library events in cedar park,library events at . less near disha,0.467138,the library events is also in a brooklyn .,0.451801,library events cedar stick on cedar breath,0.516973
4,Please timer ko rokey,please pause timer,please pause timer .,0.707107,please pause the timer .,0.622333,please pause timer remaining,0.707107
...,...,...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,concerts to get the chocolate this summer,0.516973,ski the concerts this summer,0.562341,concerts this summer,0.716531
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,i have heard my alarm for the angry of the amo...,0.244462,i am the tulsi of my alarms for the time . . ....,0.368214,i am a good song to visit the new era of sean ...,0.698534
1387,please Tupac play kare,please play tupac,tupac please play onions .,0.622333,please play tupac,1.000000,please play tupac on foods .,0.472871
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,the rhonda level message from . . she and to t...,0.549100,message rhonda . get a . with line or is there...,0.298179,fight with purple finals . . . . and king year...,0.467138


In [20]:
sentence_bleu(["i have eaten".split()], "i have eaten the food".split() )

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.5623413251903491

In [21]:
data_val['bleu_score1'].mean()

0.5133288780758203

In [22]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
data_val['model_output2']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score2']=data_val[['trr','model_output2']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [23]:
data_val['bleu_score2'].mean()

0.5150145071770685

In [24]:
a=['please pause timer','please pause timer']
print(a[0]==a[1])
sentence_bleu(list(a[0].split()),a[1].split())

True


0

In [25]:
a=data_val[['trr','model_output1']].values[4]
print(a)
print(a[0]==a[1])
print(a[0])
print(a[1])
sentence_bleu(a[0].split(),a[1].split())

['please pause timer' 'please pause timer remaining']
False
please pause timer
please pause timer remaining


0

In [26]:
from statistics import mean

In [27]:
l1=[]
for i in data_val[['trr','model_output1']].values:
    l1.append(sentence_bleu(i[0].split(),i[1].split()))
mean(l1)

0.21165887491462884

In [28]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-02-15e/translator1')
data_val['model_output3']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score3']=data_val[['trr','model_output3']].apply(lambda x: cal_bleu(*x),axis=1)

In [29]:
data_val['bleu_score3'].mean()

0.5080865740301987

In [30]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-01-15e/translator1')
data_val['model_output4']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score4']=data_val[['trr','model_output4']].apply(lambda x: cal_bleu(*x),axis=1)


In [31]:
data_val['bleu_score4'].mean()

0.48659777360983514

In [32]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-15e/translator')
data_val['model_output5']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score5']=data_val[['trr','model_output5']].apply(lambda x: cal_bleu(*x),axis=1)


In [33]:
data_val['bleu_score5'].mean()

0.5304219611631699

In [34]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-22e/translator')
data_val['model_output6']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score6']=data_val[['trr','model_output6']].apply(lambda x: cal_bleu(*x),axis=1)


In [35]:
data_val['bleu_score6'].mean()

0.5189061441730153

In [36]:
reloaded = tf.saved_model.load('/kaggle/input/translator1/translator1')
data_val['model_output7']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score7']=data_val[['trr','model_output7']].apply(lambda x: cal_bleu(*x),axis=1)


In [37]:
data_val['bleu_score7'].mean()

0.5152864642062627

In [38]:
reloaded = tf.saved_model.load('/kaggle/input/translator-2/translator1')
data_val['model_output8']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score8']=data_val[['trr','model_output8']].apply(lambda x: cal_bleu(*x),axis=1)


In [39]:
data_val['bleu_score8'].mean()

0.5088568411298185

In [40]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
reloaded.tf_translate(
                tf.constant([
                t_text('tumhara naam kya hai')
                        ]))['text'][0].numpy().decode()

'whats the name in the name ? ? ?'

In [41]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
data_val['model_output9']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score9']=data_val[['trr','model_output9']].apply(lambda x: cal_bleu(*x),axis=1)


In [42]:
data_val['bleu_score9'].mean()

0.43357829431848216

In [43]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')

three_input_text = tf.constant([#'sabse best friend', 
                                #'That toh I know',
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor([b'mr . father , the name is pink' b'you drank' b'what to do funny . . .'], shape=(3,), dtype=string)
mr . father , the name is pink
you drank
what to do funny . . .


In [44]:
reloaded = tf.saved_model.load('/kaggle/input/file-name-20-02/file_name_20_0.2/translator')
data_val['model_output10']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score10']=data_val[['trr','model_output10']].apply(lambda x: cal_bleu(*x),axis=1)


In [45]:
data_val['bleu_score10'].mean()

0.33930362439353656

In [46]:
reloaded = tf.saved_model.load('/kaggle/input/anuvaad-03/anuvaad_0.3/translator')
data_val['model_output11']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score11']=data_val[['trr','model_output11']].apply(lambda x: cal_bleu(*x),axis=1)


In [47]:
data_val['bleu_score11'].mean()

0.2523883413874968

In [48]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-cv-20-15-14e/translator1')
data_val['model_output12']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score12']=data_val[['trr','model_output12']].apply(lambda x: cal_bleu(*x),axis=1)


In [49]:
data_val['bleu_score12'].mean()

0.5125867244659322

In [50]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch/translator1')
data_val['model_output13']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score13']=data_val[['trr','model_output13']].apply(lambda x: cal_bleu(*x),axis=1)


In [51]:
data_val['bleu_score13'].mean()

0.5297200745243772

In [52]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch-v2/translator1')
data_val['model_output14']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score14']=data_val[['trr','model_output14']].apply(lambda x: cal_bleu(*x),axis=1)


In [53]:
data_val['bleu_score14'].mean()

0.5291024513431882

In [54]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch/translator1')
data_val['model_output15']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score15']=data_val[['trr','model_output15']].apply(lambda x: cal_bleu(*x),axis=1)


In [55]:
data_val['bleu_score15'].mean()

0.4990433408151966

In [56]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch-v2/translator1')
data_val['model_output16']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score16']=data_val[['trr','model_output16']].apply(lambda x: cal_bleu(*x),axis=1)


In [57]:
data_val['bleu_score16'].mean()

0.49295314640515886

In [58]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-16bs-mixed-batch/translator1')
data_val['model_output17']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score17']=data_val[['trr','model_output17']].apply(lambda x: cal_bleu(*x),axis=1)


In [59]:
data_val['bleu_score17'].mean()

0.516383365722098

In [ ]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-16bs-mixed-batch-v2/translator1')
data_val['model_output18']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score18']=data_val[['trr','model_output18']].apply(lambda x: cal_bleu(*x),axis=1)


In [ ]:
data_val['bleu_score18'].mean()

In [13]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch-v2/translator1')
data_val['model_output19']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score19']=data_val[['trr','model_output19']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [14]:
data_val['bleu_score19'].mean()

0.5474122770314485

In [15]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch/translator1')
data_val['model_output20']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score20']=data_val[['trr','model_output20']].apply(lambda x: cal_bleu(*x),axis=1)


In [16]:
data_val['bleu_score20'].mean()

0.5518800765884747

In [17]:
data_val

,Sentence,trr,model_output19,bleu_score19,model_output20,bleu_score20
0,mere timer ko roko,pause my timer .,subtract my timer,0.544446,subtract my timer,0.544446
1,Please Liz ke liye video message banaye,please make video message for liz,please send a message to peter .,0.731110,please send a message to peter .,0.731110
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,create an alarm at pm and pm on tuesday,0.385233,create an alarm at pm and pm on tuesday,0.385233
3,Library events Cedar Park me hai,library events in cedar park,library events in cedar park,1.000000,library events in cedar park,1.000000
4,Please timer ko rokey,please pause timer,please pause timer,1.000000,please pause timer,1.000000
...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,outdoor concerts this summer,1.000000,outdoor concerts this summer,1.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,alarm set for hai program . . . with which com...,0.648139,alarm set for you . . there is a little bit of...,0.744782
1387,please Tupac play kare,please play tupac,please play tupac,1.000000,please play tupac,1.000000
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,message rhonda ask her details about vacation ...,1.000000,message rhonda ask her details about vacation ...,1.000000


In [ ]:
'ok'

In [ ]:
data_val.to_csv('bleu_score.csv', index=False)